In [1]:
from marubatsu import Marubatsu
from tree import Mbtree

mbtree = Mbtree.load("../data/aidata")
bestmoves_by_board = {}
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in bestmoves_by_board.keys():
        bestmoves_by_board[txt] = node.bestmoves
        if node.mb.status == Marubatsu.PLAYING and len(node.children) != len(node.bestmoves):
            mblist_by_board.append(node.mb)

In [2]:
print(len(mblist_by_board))

3191


In [3]:
registered_boards = []
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in registered_boards:
        registered_boards.append(txt)
        if node.mb.status == Marubatsu.PLAYING and len(node.children) != len(node.bestmoves):
            mblist_by_board.append(node.mb)

In [4]:
print(len(mblist_by_board))

3191


In [5]:
registered_boards = set()
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in registered_boards:
        registered_boards.add(txt)
        if node.mb.status == Marubatsu.PLAYING and len(node.children) != len(node.bestmoves):
            mblist_by_board.append(node.mb)

In [6]:
print(len(mblist_by_board))

3191


In [7]:
import pickle
import gzip
with gzip.open("../data/mblist_by_board2.dat", "wb") as f:
    pickle.dump(mblist_by_board, f)

In [8]:
from util import Check_solved, load_bestmoves, load_mblist
from tqdm import tqdm

@staticmethod
def is_strongly_solved(ai, params=None):
    if Check_solved.bestmoves_by_board is None:
        Check_solved.bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board.dat")
    if Check_solved.mblist_by_board is None:
        Check_solved.mblist_by_board = load_mblist("../data/mblist_by_board2.dat")            
    if params is None:
        params = {}
    count = 0
    incorrectlist = []
    for mb in tqdm(Check_solved.mblist_by_board):
        candidate = set(ai(mb, candidate=True, **params))
        bestmoves = set(Check_solved.bestmoves_by_board[mb.board_to_str()])
        if candidate <= bestmoves:
            count += 1
        else:
            incorrectlist.append((mb, candidate, bestmoves))
    nodenum = len(Check_solved.mblist_by_board)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum, incorrectlist

Check_solved.is_strongly_solved = is_strongly_solved

In [9]:
from ai import ai2s, ai14s, ai_gt6
result, incorrectlist = Check_solved.is_strongly_solved(ai2s)
print(result)

100%|██████████| 3191/3191 [00:00<00:00, 3360.41it/s]

0/3191 0.00%
False


In [10]:
result, incorrectlist = Check_solved.is_strongly_solved(ai14s)
print(result)

100%|██████████| 3191/3191 [00:01<00:00, 2682.52it/s]

3139/3191 98.37%
False


In [11]:
from util import load_bestmoves
bestmoves = load_bestmoves("../data/bestmoves_by_board.dat")
result, incorrectlist = Check_solved.is_strongly_solved(ai_gt6, params={"bestmoves_by_board": bestmoves})
print(result)

100%|██████████| 3191/3191 [00:00<00:00, 110275.47it/s]

3191/3191 100.00%
True


In [12]:
mb = Marubatsu()
mb.move(1, 1)
mb.move(0, 0)
mb.move(1, 0)
print(mb)

board = []
for x in range(mb.BOARD_SIZE):
    col = []
    for y in range(mb.BOARD_SIZE):
        col.append(mb.board[y][2 - x])
    board.append(col)
mb.board = board
print(mb)

Turn x
xO.
.o.
...

Turn x
..x
.oo
...



In [13]:
print(mb.board_to_str())

....o.xo.


In [14]:
mb = Marubatsu()
mb.move(1, 1)
mb.move(0, 0)
mb.move(1, 0)

txt = ""
for x in range(mb.BOARD_SIZE):
    for y in range(mb.BOARD_SIZE):
        txt += mb.board[y][2 - x]
print(txt)

....o.xo.


In [15]:
def calc_same_boardtexts(mb):
    data = [ [ 0,  0,  1, 1, -1,  0],
             [ 1, -1,  0, 1,  0, -1],
             [ 1,  0, -1, 0,  1,  0],
             [ 1, -1,  0, 0,  0,  1],
             [ 0,  1,  0, 1,  0, -1],
             [ 1,  0, -1, 1, -1,  0],
             [ 0,  0,  1, 0,  1,  0], ]
    boardtexts = set([mb.board_to_str()])
    for xa, xb, xc, ya, yb, yc in data:
        txt = ""
        for x in range(mb.BOARD_SIZE):
            for y in range(mb.BOARD_SIZE):
                txt += mb.board[xa * 2 + xb * x + xc * y][ya * 2 + yb * x + yc * y]
        boardtexts.add(txt)
    return boardtexts

In [16]:
print(calc_same_boardtexts(mb))

{'..x.oo...', '....o.xo.', 'x..oo....', 'xo..o....', '...oo.x..', '....oo..x', '.ox.o....', '....o..ox'}


In [17]:
mb.restart() # ゲームをリセットする
print(calc_same_boardtexts(mb))

{'.........'}


In [18]:
registered_boards = set()
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in registered_boards:
        registered_boards |= calc_same_boardtexts(node.mb)
        mblist_by_board.append(node.mb)

In [19]:
print(len(mblist_by_board))

765


In [20]:
registered_boards = set()
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in registered_boards:
        registered_boards |= calc_same_boardtexts(node.mb)
        if node.mb.status == Marubatsu.PLAYING and len(node.children) != len(node.bestmoves):
            mblist_by_board.append(node.mb)

In [21]:
print(len(mblist_by_board))

431


In [22]:
with gzip.open("../data/mblist_by_board_min.dat", "wb") as f:
    pickle.dump(mblist_by_board, f)

In [23]:
@staticmethod
def is_strongly_solved(ai, params=None):
    if Check_solved.bestmoves_by_board is None:
        Check_solved.bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board.dat")
    if Check_solved.mblist_by_board is None:
        Check_solved.mblist_by_board = load_mblist("../data/mblist_by_board_min.dat")            
    if params is None:
        params = {}
    count = 0
    incorrectlist = []
    for mb in tqdm(Check_solved.mblist_by_board):
        candidate = set(ai(mb, candidate=True, **params))
        bestmoves = set(Check_solved.bestmoves_by_board[mb.board_to_str()])
        if candidate <= bestmoves:
            count += 1
        else:
            incorrectlist.append((mb, candidate, bestmoves))
    nodenum = len(Check_solved.mblist_by_board)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum, incorrectlist

Check_solved.is_strongly_solved = is_strongly_solved
Check_solved.mblist_by_board = None

In [24]:
result, incorrectlist = Check_solved.is_strongly_solved(ai2s)
print(result)

100%|██████████| 431/431 [00:00<00:00, 3734.25it/s]

0/431 0.00%
False


In [25]:
result, incorrectlist = Check_solved.is_strongly_solved(ai14s)
print(result)

100%|██████████| 431/431 [00:00<00:00, 2177.32it/s]

424/431 98.38%
False


In [26]:
for mb, candidates, best_moves in incorrectlist:
    print(mb)
    print("candidates", candidates)
    print("best_moves", best_moves)

Turn o
o.X
...
...

candidates {(1, 1)}
best_moves {(0, 1), (0, 2), (2, 2)}
Turn x
oOx
...
...

candidates {(1, 1)}
best_moves {(2, 1), (2, 2)}
Turn x
o.x
...
.O.

candidates {(1, 1)}
best_moves {(2, 2)}
Turn x
xo.
..O
...

candidates {(1, 1)}
best_moves {(0, 2)}
Turn x
.o.
x.O
...

candidates {(0, 0)}
best_moves {(2, 0), (2, 2)}
Turn o
.o.
...
X..

candidates {(1, 1)}
best_moves {(0, 0)}
Turn x
.o.
..O
x..

candidates {(1, 1)}
best_moves {(2, 2), (0, 0)}


In [27]:
result, incorrectlist = Check_solved.is_strongly_solved(ai_gt6, params={"bestmoves_by_board": bestmoves})
print(result)

100%|██████████| 431/431 [00:00<00:00, 143540.18it/s]

431/431 100.00%
True
